In [1]:
%%file InputParser.py
import xml.etree.ElementTree as ET
import numpy as np
from copy import deepcopy

class InputParser:
    
    """Returns the roots of a quadratic equation: ax^2 + bx + c = 0.
    
    INPUTS
    =======
    a: float, optional, default value is 1
       Coefficient of quadratic term
    b: float, optional, default value is 2
       Coefficient of linear term
    c: float, optional, default value is 0
       Constant term
    
    RETURNS
    ========
    roots: 2-tuple of complex floats
       Has the form (root1, root2) unless a = 0 
       in which case a ValueError exception is raised
    
    EXAMPLES
    =========
    >>> input_ = InputParser('rxns.xml')
    >>> print(input_.species)
    [ 'H' 'O' 'OH' 'H2' 'H2O' 'O2' ]
    """
    
    def __init__(self, file_name):
        self.file_name = file_name
        self.raw = ET.parse(self.file_name).getroot()
        self.species = self.raw.find('phase').find('speciesArray').text.strip().split()
        self.reactions = self.get_reactions(self.raw)
        self.nu_react, self.nu_prod = self.get_nu(self.reactions, self.species)
        self.rate_coeff_params = self.get_rate_coeff_params(self.reactions)
        
    def get_reactions(self, raw):
        
        def parse_rate_coeff(reaction, reaction_dict):
            rc_ = reaction.find('rateCoeff')
            reaction_dict['rateCoeffParams'] = dict()
            if None != rc_.find('Constant'):
                reaction_dict['rateCoeffParams']['type'] = 'Constant'
                reaction_dict['rateCoeffParams']['k'] = float(rc_.find('Constant').find('k').text)
            elif None != rc_.find('Arrhenius'):
                reaction_dict['rateCoeffParams']['type'] = 'Arrhenius'
                reaction_dict['rateCoeffParams']['A'] = float(rc_.find('Arrhenius').find('A').text)
                reaction_dict['rateCoeffParams']['E'] = float(rc_.find('Arrhenius').find('E').text)
            elif None != rc_.find('modifiedArrhenius'):
                reaction_dict['rateCoeffParams']['type'] = 'modifiedArrhenius'
                reaction_dict['rateCoeffParams']['A'] = float(rc_.find('modifiedArrhenius').find('A').text)
                reaction_dict['rateCoeffParams']['b'] = float(rc_.find('modifiedArrhenius').find('b').text)
                reaction_dict['rateCoeffParams']['E'] = float(rc_.find('modifiedArrhenius').find('E').text)
            else:
                raise NotImplementedError('The type of reaction rate coefficient has not been implemented.')
            
        reactions = []
        for i, reaction in enumerate(raw.find('reactionData')):
            reactions.append(deepcopy(reaction.attrib))
            parse_rate_coeff(reaction, reactions[i])
            reactions[i]['equation'] = reaction.find('equation').text
            reactions[i]['reactants'] = {s.split(':')[0]:float(s.split(':')[1]) \
                                         for s in reaction.find('reactants').text.split()}
            reactions[i]['products'] = {s.split(':')[0]:float(s.split(':')[1]) \
                                         for s in reaction.find('products').text.split()}
        return reactions
    
    def get_nu(self, reactions, species):
        nu_react = np.zeros((len(species), len(reactions)))
        nu_prod = np.zeros((len(species), len(reactions)))
        for i, reaction in enumerate(reactions):
            if not (reaction['reversible'] == 'no' and reaction['type'] == 'Elementary'):
                raise NotImplementedError('The type of reaction has not been implemented.')
            for specie, stoi in reaction['reactants'].items():
                nu_react[species.index(specie), i] = stoi
            for specie, stoi in reaction['products'].items():
                nu_prod[species.index(specie), i] = stoi
        return nu_react, nu_prod
    
    def get_rate_coeff_params(self, reactions):
        return [reaction['rateCoeffParams'] for reaction in reactions]
    
    def __repr__(self):
        '''Return a printable representation of the object.'''
        return 'InputParser(file_name=\'{}\')'.format(self.file_name)
    
    def __len__(self):
        '''Return the number of chemical reactions.'''
        return len(self.reactions)

Overwriting InputParser.py


In [2]:
from InputParser import *
input_ = InputParser('rxns.xml')
input_.reactions

[{'equation': 'H + O2 =] OH + O',
  'id': 'reaction01',
  'products': {'O': 1.0, 'OH': 1.0},
  'rateCoeffParams': {'A': 35200000000.0, 'E': 71400.0, 'type': 'Arrhenius'},
  'reactants': {'H': 1.0, 'O2': 1.0},
  'reversible': 'no',
  'type': 'Elementary'},
 {'equation': 'H2 + O =] OH + H',
  'id': 'reaction02',
  'products': {'H': 1.0, 'OH': 1.0},
  'rateCoeffParams': {'A': 0.0506,
   'E': 26300.0,
   'b': 2.7,
   'type': 'modifiedArrhenius'},
  'reactants': {'H2': 1.0, 'O': 1.0},
  'reversible': 'no',
  'type': 'Elementary'},
 {'equation': 'H2 + OH =] H2O + H',
  'id': 'reaction03',
  'products': {'H': 1.0, 'H2O': 1.0},
  'rateCoeffParams': {'k': 1000.0, 'type': 'Constant'},
  'reactants': {'H2': 1.0, 'OH': 1.0},
  'reversible': 'no',
  'type': 'Elementary'}]

In [3]:
print(input_.nu_react)
print(input_.nu_prod)

[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 0.  1.  1.]
 [ 0.  0.  0.]
 [ 1.  0.  0.]]
[[ 0.  1.  1.]
 [ 1.  0.  0.]
 [ 1.  1.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  1.]
 [ 0.  0.  0.]]


In [4]:
input_.rate_coeff_params

[{'A': 35200000000.0, 'E': 71400.0, 'type': 'Arrhenius'},
 {'A': 0.0506, 'E': 26300.0, 'b': 2.7, 'type': 'modifiedArrhenius'},
 {'k': 1000.0, 'type': 'Constant'}]

In [5]:
print(repr(input_))
print(len(input_))

InputParser(file_name='rxns.xml')
3


In [8]:
input_.species

['H', 'O', 'OH', 'H2', 'H2O', 'O2']